In [3]:
from sklearn.datasets import fetch_20newsgroups
import cudf
import cupy as cp
from cuBERTopic import gpu_BERTopic
from embedding_extraction import create_embeddings
data = fetch_20newsgroups(subset='all')['data']

In [4]:
gpu_topic = gpu_BERTopic()
documents = cudf.DataFrame(
    {"Document": data, "ID": cp.arange(len(data)), "Topic": None}
)

In [6]:
%%time
# Extract embeddings
embeddings = create_embeddings(
    documents.Document
)

CPU times: user 16 s, sys: 3.02 s, total: 19 s
Wall time: 11.3 s


In [7]:
%%time
# Dimensionality Reduction
umap_embeddings = gpu_topic.reduce_dimensionality(embeddings)
del embeddings

CPU times: user 304 ms, sys: 276 ms, total: 580 ms
Wall time: 581 ms


In [9]:
%%time
# Cluster UMAP embeddings with HDBSCAN
documents, probabilities = gpu_topic.clustering_hdbscan(
    umap_embeddings,
    documents
)
del umap_embeddings

Label prop iterations: 22
Label prop iterations: 6
Label prop iterations: 5
Label prop iterations: 4
Label prop iterations: 4
Iterations: 5
2003,164,398,14,290,1452
Label prop iterations: 3
Label prop iterations: 3
Iterations: 2
1106,75,194,7,112,263
CPU times: user 168 ms, sys: 305 ms, total: 473 ms
Wall time: 471 ms


In [10]:
%%time
# Topic representation
tf_idf, count, docs_per_topics_topics = gpu_topic.create_topics(
    documents
)
top_n_words, name_repr = gpu_topic.extract_top_n_words_per_topic(
    tf_idf, count, docs_per_topics_topics, n=30
)

gpu_topic.topic_sizes_df["Name"] = gpu_topic.topic_sizes_df["Topic"].map(
    name_repr
)

CPU times: user 11 s, sys: 242 ms, total: 11.3 s
Wall time: 11.3 s
